In [ ]:
!pip install -q -U transformers peft accelerate optimum
!pip install -q datasets==2.15.0

In [ ]:
!pip install -q auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/

In [ ]:
import torch, auto_gptq
from PIL import Image
from transformers import AutoModel, AutoTokenizer
from auto_gptq.modeling import BaseGPTQForCausalLM

In [ ]:
auto_gptq.modeling._base.SUPPORTED_MODELS = ["internlm"]
torch.set_grad_enabled(False)

In [ ]:
class InternLMXComposer2QForCausalLM(BaseGPTQForCausalLM):
    layers_block_name = "model.layers"
    outside_layer_modules = [
        'vit', 'vision_proj', 'model.tok_embeddings', 'model.norm', 'output',
    ]
    inside_layer_modules = [
        ["attention.wqkv.linear"],
        ["attention.wo.linear"],
        ["feed_forward.w1.linear", "feed_forward.w3.linear"],
        ["feed_forward.w2.linear"],
    ]

In [ ]:
model = InternLMXComposer2QForCausalLM.from_quantized(
  'internlm/internlm-xcomposer2-7b-4bit', trust_remote_code=True, device="cuda:0").eval()
tokenizer = AutoTokenizer.from_pretrained(
  'internlm/internlm-xcomposer2-7b-4bit', trust_remote_code=True)

In [ ]:
# from datasets import load_dataset

# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
image_names=['/content/OR2.PNG','']

In [ ]:
for image_name in image_names:
  img_path_list = [
    image_name,
  ]
  images = []
  for img_path in img_path_list:
    image = Image.open(img_path).convert("RGB")
    image = model.vis_processor(image)
    images.append(image)
    image = torch.stack(images)
  query = f'<ImageHere> <ImageHere> please explain the image.the image is based on {image_name}'
  with torch.cuda.amp.autocast():
    response, history = model.chat(tokenizer, query=query, image=image, history=[], do_sample=False)
  print("-------------------------------------------------------------------------------------------------------------------------------------------------")
  print(response)


